# Oefening deel 2: COP van een warmtepomp bepalen

> **Workshop Atic 4D**  
> **Lesgevers:** Lien De Backer & Jakob De Vreese

In deze oefening focussen we op de 4-pijps warmtepomp van ons gebouw. Hiervoor hebben we 4 datasets ter beschikking.

## Datasets

1) Elektrisch verbruik (2022 tot circa nu)
2) Warmte opwekking (2022 tot circa nu)
3) Warmte naar BEO-veld (2022 tot circa nu)
4) Warmte uit BEO-veld (2022 tot circa nu)

## Doel van de oefening

In dit deel van de oefening gaan we de COP van onze warmtepomp bepalen, het optimale rendement zoeken, en visualisaties maken.

### 🎯 Leerdoelen

Na deze oefening kun je:

- Meerdere energiestromen combineren en synchroniseren
- COP berekenen op verschillende tijdsschalen
- Data-kwaliteitsproblemen identificeren en aanpakken
- Energiebalansen opstellen en valideren
- Relaties met externe factoren (temperatuur) onderzoeken

### Structuur

- Packages importeren
- Data inladen en verkennen
- Data opschonen en voorbereiden
- Energiestromen combineren en valideren
- COP berekeningen per periode
- Visualisatie en analyse van COP-trends
- Relatie COP met externe factoren (proberen)
- Energiebalans BEO-veld
- Conclusies en aanbevelingen

## 📐 Theoretische achtergrond

### COP (Coefficient of Performance)

$$COP_{verwarming} = \frac{Q_{warmte,netto}}{W_{elektrisch}}$$

Waarbij:
- $Q_{warmte,netto}$ = warmte-output (kWh) - injectie gasketel
- $W_{elektrisch}$ = elektrisch verbruik (kWh)

Een typische warmtepomp heeft:
- **COP 3-5** bij gunstige condities (milde buitentemperatuur)
- **COP 2-3** bij koude condities
- **COP < 2** wijst op problemen

### Energiebalans BEO-veld

$$\Delta E_{BEO} = E_{onttrokken} - E_{geïnjecteerd}$$

Een gezond BEO-veld is in balans over het jaar (zomer: laden, winter: ontladen).

In [ ]:
# VUL AAN: importeer de libraries

## Stap 1: Data inladen

We laden volgende datasets in en _onderzoeken de data_:
- Elektrisch verbruik warmtepomp
- Warmte opwekking warmtepomp
- Koude opwekking warmtepomp
- Warmte naar BEO-veld
- Warmte uit BEO-veld

Hieronder krijg je een hulpfunctie om de data schoon in te laden van sommige datasets (wanneer aangegeven)

In [ ]:
from google.colab import drive
drive.mount('/content/drive/workshop_atic-4D_25')

In [ ]:
# Herbruikbare functie voor het inladen van GBS-export bestanden
def load_gbs_export(filepath, energy_cols_keywords, date_format='%m/%d/%y'):
    """
    Laad een tab-gescheiden GBS export bestand in.
    
    Parameters:
    -----------
    filepath : str
        Pad naar het bestand
    energy_cols_keywords : dict
        Dict met {nieuwe_naam: zoekterm} voor kolommen
    date_format : str
        Format van de datum in het bestand
        
    Returns:
    --------
    pd.DataFrame met DatetimeIndex en hernoemde kolommen
    """
    # Inladen
    df = pd.read_csv(filepath, sep='\t')
    
    # Verwijder Unnamed kolommen
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    
    # Zoek en hernoem energie kolommen
    rename_dict = {}
    for new_name, keyword in energy_cols_keywords.items():
        matching_cols = [c for c in df.columns if keyword.lower() in c.lower()]
        if matching_cols:
            rename_dict[matching_cols[0]] = new_name
    
    df = df.rename(columns=rename_dict)
    
    # Vind Time kolom
    time_col = 'Time' if 'Time' in df.columns else [c for c in df.columns if 'time' in c.lower()][0]
    
    # Parse datum (alleen deel voor komma)
    df['date'] = (
        df[time_col]
        .astype(str)
        .str.split(',', n=1)
        .str[0]
        .str.strip()
    )
    
    df['date'] = pd.to_datetime(df['date'], format=date_format, errors='coerce')
    df = df.dropna(subset=['date'])
    
    # Zet index en drop originele time kolom
    df = df.set_index('date').drop(columns=[time_col])
    
    # Selecteer alleen de hernoemde kolommen
    valid_cols = [col for col in energy_cols_keywords.keys() if col in df.columns]
    
    return df[valid_cols]

print("✓ Hulpfuncties gedefinieerd")

### Elektrisch verbruik

Het elektrisch verbruik is handmatig uitgelezen en slechts ongeveer jaarlijks (onregelmatig) geregistreerd.

**Locatie**: `data/elek_WP_1102.csv`  
**Kolommen**: `datum`, `teller_30P1`  

In [ ]:
# 2.1 Elektrisch verbruik warmtepomp (handmatig afgelezen)
elek = pd.read_csv('data/elek_WP_1102.csv', parse_dates=['datum'], index_col='datum')
elek.columns = elek.columns.str.replace('\ufeff', '', regex=False).str.strip()
elek = elek.sort_index()

print(f"📌 Elektrisch verbruik: {len(elek)} metingen van {elek.index.min():%Y-%m-%d} tot {elek.index.max():%Y-%m-%d}")
elek.head()

In [ ]:
# VUL AAN: neem snel een kijkje en plot elek!


### Warmte- en koude-opwekking voorbereiden

Je werkt met een export uit het gebouwbeheersysteem (calorieteller van de warmtepomp aan de verbruikerszijde).  
**Bestand**: `data/calorieteller_wp.txt` (tab-gescheiden)  
**Kolommen**:
- Time: datum + tijd + tijdzone als tekst (bv. 5/29/20, 12:00:00 AM CEST)
- Twee zeer lange kolomnamen met cumulatieve energietellers (warm en koud) in kWh (let op: header bevat zowel MWh-tekst als kWh-eenheid).
- Eventueel een lege/Unnamed kolom.

> TIP! -> hier kunnen we de hulpfunctie gebruiken!

In [ ]:
# 2.2 Warmte/koude opwekking warmtepomp
# VUL AAN: maak een variabele calorieteller_wp, zet de kolommen gelijk aan energieverbruik_warm en energieverbruik_koud. De data staat in data/calorieteller_wp.txt


# VUL AAN: print het aantal dagen die in de dataset voorkomen

# VUL AAN: toon de eerste lijnen van de dataset


Neem een snel kijkje!

In [ ]:
# VUL AAN: plot de dataset


#### Vragen voor reflectie

- Waarom is het nuttig de kolomnamen robuust te vinden i.p.v. hard te typen?
- Wat gebeurt er als je de tijdzone en tijd niet wegknipt?
- Wat is het verschil tussen cumulatieve meter en dagelijkse productie?

### Injectie gasketel dataset

Bekijk de kolomnamen in `data/calorieteller_injectie_LT.txt` en probeer de data in te laden met de hulpfunctie!

In [ ]:
# 2.3 Injectie gasketel (backup warmte)
# VUL AAN: doe het zelfde voor cal_injectie, de data staat 'data/calorieteller_injectie_LT.txt' en 'injectie_warm': 'energieverbruik warm', print het aantal dagen en toon de dataset.



In [ ]:
# VUL AAN: plot de dataset


### Calorieteller Beoveld inladen

Ga naar `data/calorieteller_beo.txt` en probeer de data correct in te laden!

In [ ]:
# 2.4 BEO-veld (warmte in/uit bodem)
# VUL AAN: maak een variabele cal_beo, data staat 'data/calorieteller_beo.txt' en de keywords zijn: 'beo_warm': 'warm', 'beo_koud': 'koud'

# Converteer van MWh naar kWh (indien nodig) 
if cal_beo['beo_warm'].max() < 1000:  # Check of data in MWh staat
    cal_beo['beo_warm'] *= 1000
    cal_beo['beo_koud'] *= 1000

# VUL AAN: Print het aantal dagen, en print de eerste lijnen van de dataset



In [ ]:
# VUL AAN: Plot de dataset

## 🔍 2. Data verkennen en visualiseren

Nu gaan we de data verkennen, naast elkaar plotten en reflecteren. 
We onderscheiden eenerzeids de cumulatieve meters, en anderzeids de dagelijkse verbruiken. 

In [ ]:
# 3.1 Overzicht cumulatieve meters
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# VUL AAN: Zet de opwekking_warm, opwekking_koud, injectie_warm en in en uit beoveld samen elk op hun eigen subplot


for ax in axes.flat:
    ax.set_ylabel('kWh (cumulatief)')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

De **koude-opwekking** in ons systeem wordt _rechtstreeks_ verzorgd door het BEO-veld via een _platenwisselaar_. Hierbij komt geen warmtepomp kijken, wat betekent dat het _theoretische rendement_ gelijk zou moeten zijn aan ongeveer 1. 

Voor de **warmte-opwekking** is de situatie anders. Hier gebruiken we een warmtepomp, de output moet dus hoger zijn dan wat we uit het BEO-veld halen, en de factor is dan onze `COP`. Dit is een eerste manier hoe we de COP kunnen berekenen. 

Laten we eerst visualisaties maken om onze trend een eerste keer te analyseren, en patronen te proberen herkennen.

In [ ]:
# VUL AAN: maak een plot waarbij we de output van de warmtepomp en de injectie van koude in het beoveld tegenover elkaar zetten.


In [ ]:
# VUL AAN: Doe nu hetzelfde voor koud vs warm (dit is met platenwisselaar, we zouden ongeveer een rendement van +- 90% moeten zien -> is dit zo?)


## 3. 🧮 Berekening COP

De wet van behoud van energie stelt dat de energie die je uit het beo-veld haalt, plus de elektrische energie die je in de warmtepomp steekt gelijk moet zijn aan de warmte die je uit de warmtepomp haalt. Ofwel wiskundig:

$$
Q_{beo} + Q_{elek} = Q_{wp}
$$

$$
\frac{Q_{wp}}{Q_{elek}} = COP
$$

ofwel:

$$
\frac{Q_{beo} + Q_{elek}}{Q_{elek}} = COP
$$

Bovenstaande doen we, omdat er sterk vermoeden is dat de warmte die we uit de warmtepomp halen sterk bevuild is doordat de pomp draait, de warmtepomp uit, en de ketel warmte injecteert.

Dit houd wel geen rekening met elektrische verliezen van de warmtepomp. Maar laten we eens de COP zo berekenen.


In [ ]:
# VUL AAN: maak een variabele totaal_elek en zet deze gelijk aan de laatste tellerstand (maximale getal)

# VUL AAN: maak een variabele totaal_injectie_koud_beo en zet ook deze gelijk aan de totaal opgewekte koude naar het beoveld

# VUL AAN: maak een variabele cop_tot en zet deze gelijk aan de berekening van de cop met de Q_beo en Q_elek als waarden


print(f'COP-berekening aan de hand elektrisch verbruik en beo injectie koud')
print()
print(f'Totaal elektrisch verbruik: {totaal_elek} kwh')
print(f'Totaal geinjecteerde koude beo: {totaal_injectie_koud_beo} kwh')
print()
print(f'COP: {cop_tot:.2f}')

Volgens de technische fiche van de warmtepomp zouden we een SCOP van `4.4` moeten halen. Lukt dit?

### COP evolutie

Bekijk of er een evolutie in de COP zichtbaar is. Omdat we slechts weinig meetpunten voor elektrisch verbruik hebben, aggregeren we voor elk interval tussen twee elektriciteitsmeter‑aflezingen de dagelijkse BEO‑waarden: we sommeren de energie van het BEO‑veld binnen dat interval en koppelen die som aan de bijbehorende elektriciteitsmeting.

In [ ]:
cop = elek.sort_index().copy()
cop['elektrisch_verbruik'] = cop['teller_30P1'].diff()
cop['beo_koud'] = np.nan

cal_beo['beo_koud_daily'] = cal_beo['beo_koud'].diff()

for i in range(1, len(cop)):
    d0 = cop.index[i-1]
    d1 = cop.index[i]
    mask = (cal_beo.index > d0) & (cal_beo.index <= d1)     # excl start, incl end
    cop.iloc[i, cop.columns.get_loc('beo_koud')] = cal_beo.loc[mask, 'beo_koud_daily'].sum()

# VUL AAN: voeg een kolom cop toe en zet deze gelijk aan de berekende cop volgens Q_beo en Q_elek


# VUL AAN: print de eerste lijnen van onze dataset


In [ ]:
# VUL AAN: plot de evolutie van de cop


## ⚖️ 4. Energiebalans BEO-veld

Een belangrijke factor bij BEO-velden om over langere tijd in de gaten te houden is of dit in balans blijft. Laten we dit eens voor onze installatie visualiseren, en een functie maken om het automatisch te berekenen bij andere installaties!

In [ ]:
# 8.1 Bereken dagelijkse energiestromen BEO-veld
cal_beo['beo_warm_daily'] = # VUL AAN
cal_beo['beo_koud_daily'] = # VUL AAN
cal_beo['beo_balans_daily'] = # VUL AAN

# Cumulatieve balans
cal_beo['beo_balans_cumul'] = cal_beo['beo_balans_daily'].cumsum()

# 8.2 Visualisatie energiebalans
# VUL AAN: maak 3 grafieken
# TODO -> subplot aanmaken met 3 grafieken

# Grafiek 1: Dagelijkse in/uit stromen
# TODO -> plot aanmaken van de beo_warm_daily en beo_koud_daily


# Grafiek 2: Dagelijkse netto balans
# TODO -> plot aanmaken van de balans


# Grafiek 3: Cumulatieve balans (gezondheid bodem)
# TODO -> plot aanmaken van de cumulsum

# VUL AAN: maak een tight_layout

# VUL AAN: toon de plot plt


# 8.3 Statistieken
print("="*60)
print(" "*15 + "BEO-VELD BALANS ANALYSE")
print("="*60)
print(f"\n📊 Totalen (volledige periode)")
print(f"{'─'*60}")
print(f"Totaal naar bodem (laden):     ### VUL AAN ### kWh")
print(f"Totaal uit bodem (ontladen):   ### VUL AAN ### kWh")
print(f"Netto balans:                  ### VUL AAN ### kWh")

# VUL AAN: maak een variabele balans_pct die het percentage berekend van de balans tov de totaal opgewekte koude (vergeet niet * 100 te doen op het einde!)

print(f"Balans (%):                    ### VUL AAN ### %")

print(f"\n💡 Interpretatie")
print(f"{'─'*60}")
if abs(balans_pct) < 5:
    print("✅ Uitstekend! BEO-veld is goed in balans (< 5%)")
elif abs(balans_pct) < 15:
    print("⚠️  Redelijk. Lichte onevenwichtigheid (5-15%)")
else:
    print("❌ Zorgelijk! Significante onevenwichtigheid (> 15%)")
    
if balans_pct > 0:
    print("   → Bodem warmt op (meer laden dan ontladen)")
    print("   → Risico op verminderde koelprestaties in de zomer")
else:
    print("   → Bodem koelt af (meer ontladen dan laden)")
    print("   → Risico op verminderde verwarmingsprestaties in de winter")

print(f"\n{'='*60}\n")

## 📝 9. Conclusies en aanbevelingen

Een belangrijke tool voor data analysten zijn eigen scriptjes die snel inzichten kunnen geven in specifieke datasets, met herbruikbare code. Dit is afhankelijk van het veld waarin je werkt, of van eigen accenten. Hieronder kan je een voorbeeld vinden van zo herbruikbare scripten!

In [ ]:
# 9.1 Samenvatting
print("="*80)
print(" "*25 + "SAMENVATTING COP ANALYSE")
print("="*80)

print(f"\n📊 PRESTATIE-INDICATOREN")
print(f"{'─'*80}")
print(f"Gemiddelde COP (alle metingen):           {cop['cop'].mean():.2f}")
print(f"Mediaan COP:                              {cop['cop'].median():.2f}")
print(f"Beste COP gemeten:                        {cop['cop'].max():.2f}")
print(f"Slechtste COP gemeten:                    {cop['cop'].min():.2f}")

# Aanbevelingen op basis van COP
gemiddelde_cop = cop['cop'].mean()
if gemiddelde_cop >= 4.0:
    print("✅ COP is uitstekend (≥4.0). Systeem werkt optimaal.")
elif gemiddelde_cop >= 2.5:
    print("⚠️  COP is redelijk (2.5-4.0). Er is ruimte voor verbetering:")
    print("   • Check regelinstellingen warmtepomp")
    print("   • Onderzoek of aanvoertemperaturen verlaagd kunnen worden")
else:
    print("❌ COP is onder verwachting (<2.5). Actie vereist:")
    print("   • Technische inspectie warmtepomp noodzakelijk")
    print("   • Check koudemiddel niveau en drukken")
    print("   • Verificeer compressor werking")

print(f"\n📊 BEO-VELD BALANS")
print(f"{'─'*80}")
print(f"Totaal naar bodem (laden):                {cal_beo['beo_warm'].max():.2f} kWh")
print(f"Totaal uit bodem (ontladen):              {cal_beo['beo_koud'].max():.2f} kWh")
print(f"Netto balans:                             {cal_beo['beo_warm'].max() - cal_beo['beo_koud'].max():.2f} kWh")

# Aanbevelingen BEO
beo_balans_totaal = cal_beo['beo_warm'].max() - cal_beo['beo_koud'].max()
if abs(beo_balans_totaal) > 5000:
    print(f"\n⚠️  BEO-veld onevenwichtig ({beo_balans_totaal/1000:.1f} MWh):")
    if beo_balans_totaal > 0:
        print("   • Bodem warmt op → verhoogt risico op prestatievermindering")
        print("   • Evalueer of regeneratie van bodemveld nodig is")
    else:
        print("   • Bodem koelt af → verhoogt energieverbruik")
        print("   • Overweeg actieve kloeling in de zomer om te balanceren")

print(f"\n📊 ENERGIE-OVERZICHT (totaal)")
print(f"{'─'*80}")
print(f"Elektrisch verbruik WP:                      {elek['teller_30P1'].max()} kWh")
print(f"Geproduceerde warmte WP:                     {calorieteller_wp['opwekking_warm'].max()} kWh")
print(f'Warmteproductie BEO-veld:                    {cal_beo['beo_koud'].max()} kWh')
print(f"Koudeproductie Beo-veld:                     {cal_beo['beo_warm'].max()} kWh")

-----
EINDE

------